<a href="https://colab.research.google.com/github/tashir0605/SentimentSphere/blob/main/Human_Emotion_through_Visual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [77]:
import numpy as np
import os
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Layer Definitions

In [78]:
class Dense:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size) * 0.01
        self.bias = np.random.randn(output_size, 1) * 0.01

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

# Convolutional Layer
from scipy import signal

class Convolutional:
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient

    def get_parameters(self):
        # Return kernels and biases as separate arrays
        return self.kernels, self.biases

    def set_parameters(self, params):
        if len(params) == 2:
            self.kernels, self.biases = params
        else:
            raise ValueError(f"Expected 2 parameters (kernels, biases), but got {len(params)}")

# Reshape Layer
class Reshape:
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

# Activation Functions

In [79]:
class Sigmoid:
    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, output_gradient, learning_rate):
        # Reshape output_gradient to match self.output's shape if necessary
        if output_gradient.shape != self.output.shape:
            output_gradient = output_gradient.reshape(self.output.shape)
        return output_gradient * (self.output * (1 - self.output))

# Loss Functions

In [80]:
def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

# Training Utilities

In [81]:
def train(network, loss, loss_prime, x_train, y_train, epochs=1000, learning_rate=0.01):
    for epoch in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # Forward pass
            output = x
            for layer in network:
                output = layer.forward(output)

            # Loss
            error += loss(y, output)

            # Backward pass
            grad = loss_prime(y, output)
            for layer in reversed(network):
                if hasattr(layer, 'backward'):
                    grad = layer.backward(grad, learning_rate)

        error /= len(x_train)
        print(f"{epoch + 1}/{epochs}, error={error:.4f}")

# Model Save/Load Functions

In [82]:
def save_model(network, filename="model_params.npz"):
    params = {}
    for i, layer in enumerate(network):
        if hasattr(layer, 'get_parameters'):
            # Get kernels and biases separately
            kernels, biases = layer.get_parameters()
            # Store them with separate keys
            params[f'layer_{i}_kernels'] = kernels
            params[f'layer_{i}_biases'] = biases
    np.savez(filename, **params)

def load_model(network, filename="model_params.npz"):
    if not os.path.exists(filename):
        print(f"Warning: Model file '{filename}' not found. Starting with a new model.")  # Add this line
        return False # Add this line to indicate load failure and prevent further execution
    loaded = np.load(filename, allow_pickle=True)
    for i, layer in enumerate(network):
        if hasattr(layer, 'set_parameters'):
            # Load kernels and biases using separate keys
            kernels_key = f'layer_{i}_kernels'
            biases_key = f'layer_{i}_biases'
            if kernels_key in loaded and biases_key in loaded:
                layer.set_parameters((loaded[kernels_key], loaded[biases_key]))
    return True # Indicate successful load

# Data Preprocessing

In [83]:
def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

# Network Definition & Training

In [84]:
# Define network
network = [
    Convolutional((1, 28, 28), 3, 5),  # Input shape: (depth, height, width)
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),  # Flatten output
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

# Testing

In [85]:
# Train or load
if not load_model(network):
    train(
        network,
        binary_cross_entropy,
        binary_cross_entropy_prime,
        x_train,
        y_train,
        epochs=20,
        learning_rate=0.1
    )
    save_model(network)

# Test
correct = 0
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    pred = np.argmax(output)
    true = np.argmax(y)
    if pred == true:
        correct += 1
    print(f"Predicted: {pred}, True: {true}")

print(f"Accuracy: {correct/len(x_test)*100:.2f}%")

1/20, error=0.6806
2/20, error=0.2156
3/20, error=0.0674
4/20, error=0.0539
5/20, error=0.0097
6/20, error=0.0048
7/20, error=0.0033
8/20, error=0.0026
9/20, error=0.0022
10/20, error=0.0019
11/20, error=0.0016
12/20, error=0.0014
13/20, error=0.0013
14/20, error=0.0012
15/20, error=0.0011
16/20, error=0.0010
17/20, error=0.0009
18/20, error=0.0009
19/20, error=0.0008
20/20, error=0.0008
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, T